In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                .appName("BatchAnalysis")\
                .config("spark.driver.memory", "12g")\
                .getOrCreate()

df = spark.read.csv("./datasets/2019-Nov.csv", header=True, inferSchema=True)


Phân tích tỉ lệ người dùng xem hàng rồi mua sản phẩm, người dùng xem không mua, người dùng thêm giỏ hàng

In [4]:
view_events = df.filter(df.event_type == "view")
cart_events = df.filter(df.event_type == "cart")
purchase_events = df.filter(df.event_type == "purchase")

In [7]:
condition = (view_events.user_id == purchase_events. user_id) & (view_events.product_id == purchase_events.product_id)
view_and_purchase = view_events.join(purchase_events,condition,"left_semi")\
                                .distinct()
view_and_purchase.show(10)

total_view = view_events.count()
users_view_and_purchase = view_and_purchase.count()
ratio_view_and_purchase = users_view_and_purchase / total_view
print(f"Tỉ lệ người dùng xem và mua sản phẩm: {ratio_view_and_purchase:.10f}")

+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|2019-11-14 14:57:26|      view|  17800204|2053013559868129947|   computers.desktop|   zeta| 33.44|474593321|4c95a39e-9262-44f...|
|2019-11-11 21:08:04|      view|   1004833|2053013555631882655|electronics.smart...|samsung|169.17|510670517|75a57afc-21d9-4a7...|
|2019-11-20 15:58:46|      view|   1004833|2053013555631882655|electronics.smart...|samsung| 170.3|510670517|811791b0-dcf5-4ef...|
|2019-11-21 17:19:37|      view|   1004833|2053013555631882655|electronics.smart...|samsung|170.25|510670517|43f3cdd7-7e81-4bd...|
|2019-11-12 12:59:47|      view|   5100576|2053013553341792533|  electronics.clocks

In [6]:
view_not_purchase = view_events.join(purchase_events, condition, "left_anti")\
                            .distinct()

users_view_not_purchase = view_not_purchase.count()
ratio_view_not_purchase = users_view_not_purchase / total_view
print(f"Tỉ lệ người dùng xem và không mua sản phẩm: {ratio_view_not_purchase:.10f}")

Tỉ lệ người dùng xem và không mua sản phẩm: 0.9362615962


In [8]:
condition2 = (view_events.user_id == cart_events. user_id) & (view_events.product_id == cart_events.product_id)
view_and_add_to_cart = view_events.join(cart_events,condition2,"inner")\
                                    .distinct()

users_view_and_add_to_cart = view_and_add_to_cart.count()
ratio_view_and_add_to_cart = users_view_and_add_to_cart / total_view
print(f"Tỉ lệ người dùng xem và thêm vào giỏ hàng: {ratio_view_and_add_to_cart:.10f}")

Tỉ lệ người dùng xem và thêm vào giỏ hàng: 0.3820637701
